In [3]:
import numpy as np
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install -qq langdetect
from langdetect import detect
import string
!pip install -qq contractions
!pip install -qq demoji
import demoji
demoji.download_codes()
!pip install -qq datasets

<ipython-input-1-28a5514cb77a>:7: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [2]:
from datasets import load_dataset

dataset = load_dataset("offenseval_dravidian", "tamil")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/35139 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4388 [00:00<?, ? examples/s]

In [4]:
df_train = pd.DataFrame(dataset["train"])
label_mapping = {
    0: 'Not_offensive',
    1: 'Offensive',
    2: 'Offensive',
    3: 'Offensive',
    4: 'Offensive',
    5: 'not-Tamil',
}
df_train['label'].replace(label_mapping, inplace=True)

label_mapping1 = {
    'Not_offensive':0,
    'Offensive':1,
    'not-Tamil':2,
}
df_train['label'].replace(label_mapping1, inplace=True)

print(df_train.info())
label_counts = df_train['label'].value_counts()
print(label_counts)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35139 entries, 0 to 35138
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    35139 non-null  object
 1   label   35139 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 549.2+ KB
None
0    25425
1     8260
2     1454
Name: label, dtype: int64


In [5]:
df_train.head(20)

,text,label
0,movie vara level la Erika poguthu,0
1,I love Ajith Kumar Vivegam movie inki mjy bht ...,2
2,Padam nalla comedy padama irukum polaye..,0
3,karthick subburaj anne .... intha padam vetri ...,0
4,கவுண்டர் தேவர்.சார்பாக வெற்றி பெற வாழ்த்துக்கள் 🦁,0
5,ippo intha trailer ah parkuravana oru like pod...,0
6,En thalaivan yogi babu irukkaar. Padam vera le...,0
7,Nerkonda parvai... Sema sema sema trailer,0
8,ஏய் இது 96 யார் ஏமாத்தறீங்க? செம பின்னிட்டீங்க,0
9,படம் கண்டிப்பாக வெற்றி பெற வேண்டும் செம்ம vara...,0


In [6]:
import sys
sys.path.append("/content/drive/MyDrive/Capstone/Translator/")
from google_trans_new import google_translator

In [8]:
translator = google_translator()
for index, row in df_train.iterrows(): # Check if emoji is contained in text
    text = str(row['text'])
    found = demoji.findall(text)
    if found:
        # There is an emoji now detect language
        if detect(text) == 'ta': # Text is Tamil
            for key, value in found.items():
                emojiEng = value
                # Translate to Tamil
                transText = translator.translate(emojiEng, lang_tgt='ta', lang_src='en')
                text = text.replace(key, transText + ' ')
        else: # Text is english
            for key, value in found.items():
                text = text.replace(key, value + ' ')

        df_train.at[index, 'text'] = text

In [9]:
df_train.head(20)

,text,label
0,movie vara level la Erika poguthu,0
1,I love Ajith Kumar Vivegam movie inki mjy bht ...,2
2,Padam nalla comedy padama irukum polaye..,0
3,karthick subburaj anne .... intha padam vetri ...,0
4,கவுண்டர் தேவர்.சார்பாக வெற்றி பெற வாழ்த்துக்கள...,0
5,ippo intha trailer ah parkuravana oru like pod...,0
6,En thalaivan yogi babu irukkaar. Padam vera le...,0
7,Nerkonda parvai... Sema sema sema trailer,0
8,ஏய் இது 96 யார் ஏமாத்தறீங்க? செம பின்னிட்டீங்க,0
9,படம் கண்டிப்பாக வெற்றி பெற வேண்டும் செம்ம vara...,0


In [9]:
for index, row in df_train.iterrows():
    found = demoji.findall(row['text'])
    if found:
        print('Found exception')

In [10]:
df_train.head(20)

,text,label
0,movie vara level la Erika poguthu,0
1,I love Ajith Kumar Vivegam movie inki mjy bht ...,2
2,Padam nalla comedy padama irukum polaye..,0
3,karthick subburaj anne .... intha padam vetri ...,0
4,கவுண்டர் தேவர்.சார்பாக வெற்றி பெற வாழ்த்துக்கள...,0
5,ippo intha trailer ah parkuravana oru like pod...,0
6,En thalaivan yogi babu irukkaar. Padam vera le...,0
7,Nerkonda parvai... Sema sema sema trailer,0
8,ஏய் இது 96 யார் ஏமாத்தறீங்க? செம பின்னிட்டீங்க,0
9,படம் கண்டிப்பாக வெற்றி பெற வேண்டும் செம்ம vara...,0


In [11]:
import contractions
# contracted text
for index, row in df_train.iterrows():# creating an empty list
    expanded_words = []
    for word in str(row['text']).split():# using contractions.fix to expand the shotened words
        expanded_words.append(contractions.fix(word))

    expanded_text = ' '.join(expanded_words)
    df_train.at[index, 'text'] = expanded_text


In [12]:
for index, row in df_train.iterrows():
    Sentence = str(row['text'])
    for i in Sentence:
        # checking whether the char is punctuation.
        if i in string.punctuation:
            Sentence = Sentence.replace(i, '')
    df_train.at[index, 'text'] = Sentence.lower()

In [13]:
df_train.head(20)

,text,label
0,movie vara level la erika poguthu,0
1,i love ajith kumar vivegam movie inki mjy bht ...,2
2,padam nalla comedy padama irukum polaye,0
3,karthick subburaj anne intha padam vetri aday...,0
4,கவுண்டர் தேவர்சார்பாக வெற்றி பெற வாழ்த்துக்கள்...,0
5,ippo intha trailer ah parkuravana oru like pod...,0
6,en thalaivan yogi babu irukkaar padam vera lev...,0
7,nerkonda parvai sema sema sema trailer,0
8,ஏய் இது 96 யார் ஏமாத்தறீங்க செம பின்னிட்டீங்க,0
9,படம் கண்டிப்பாக வெற்றி பெற வேண்டும் செம்ம vara...,0


In [14]:
!pip install -qq advertools
import advertools as adv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.1/312.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.4/286.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 9.9 MB/s eta 0:00:00


In [15]:
stopwords = adv.stopwords['tamil']
data=[]
for index, row in df_train.iterrows():
    sent = ''
    line = str(row['text'])
    for i in line.split():
        if i not in stopwords:
            sent += i + " "
    data.append({'text': sent, 'label': str(row['label'])})


new_df_train = pd.DataFrame(data)
new_df_train

,text,label
0,movie vara level la erika poguthu,0
1,i love ajith kumar vivegam movie inki mjy bht ...,2
2,padam nalla comedy padama irukum polaye,0
3,karthick subburaj anne intha padam vetri adaya...,0
4,கவுண்டர் தேவர்சார்பாக வெற்றி பெற வாழ்த்துக்கள்...,0
...,...,...
35134,trending number 2 idhukku nammalam karanamnu s...,0
35135,movie script super athuvum hip hop tamizha mus...,0
35136,just 3k likes for 300k likes,0
35137,aaloo le lo kanda le lo,2


In [16]:
!pip install -qq indic-nlp-library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 12.2 MB/s eta 0:00:00


In [17]:
!pip install -qq tqdm

In [18]:
from indicnlp import common
from indicnlp import loader
from indicnlp.script import indic_scripts
from indicnlp.tokenize import indic_tokenize
from indicnlp.morph import unsupervised_morph

In [19]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB | 21.96 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Updating files: 100% (28/28), done.


In [20]:
INDIC_NLP_RESOURCES = r"/content/indic_nlp_resources"
common.set_resources_path(INDIC_NLP_RESOURCES)
loader.load()
analyzer = unsupervised_morph.UnsupervisedMorphAnalyzer('ta')

INFO:morfessor.io:Loading model from '/content/indic_nlp_resources/morph/morfessor/ta.model'...
INFO:morfessor.io:Reading segmentations from '/content/indic_nlp_resources/morph/morfessor/ta.model'...
INFO:morfessor.io:Detected utf-8 encoding
INFO:morfessor.io:Done.
INFO:morfessor.io:/content/indic_nlp_resources/morph/morfessor/ta.model was read as a segmentation


In [21]:
indic_string = 'அவமரியாதை'

analyzes_tokens = analyzer.morph_analyze_document(indic_string.split(' '))
print(analyzes_tokens)

['அவமரியாதை']


In [22]:
from langdetect import detect, LangDetectException

In [23]:
from tqdm import tqdm

def stem_tamil_text(text):
    tokens = text.split()
    stemmed_tokens = []
    for token in tokens:
        analyzed_tokens = analyzer.morph_analyze_document([token])
        if analyzed_tokens:
            stemmed_tokens.append(analyzed_tokens[0])
        else:
            stemmed_tokens.append(token)  # If stemming fails, keep the original token
    return ' '.join(stemmed_tokens)

# Assuming new_df_train is your DataFrame containing text data
# Add tqdm to the loop to track progress
for index, row in tqdm(new_df_train.iterrows(), total=len(new_df_train)):
    text = str(row['text'])

    try:
        detected_lang = detect(text)
    except LangDetectException as e:
        # Handle the exception, for example by setting the detected_lang to None
        detected_lang = None

    if detected_lang == 'ta':
        stemmed_text = stem_tamil_text(text)
        new_df_train.at[index, 'text'] = stemmed_text


print(new_df_train.head(20))


100%|██████████| 35139/35139 [08:16<00:00, 70.77it/s]

                                                 text label
0                  movie vara level la erika poguthu      0
1   i love ajith kumar vivegam movie inki mjy bht ...     2
2            padam nalla comedy padama irukum polaye      0
3   karthick subburaj anne intha padam vetri adaya...     0
4        கவுண்டர் தேவர் வெற்றி பெற வாழ்த்துக் சிங்கம்     0
5   ippo intha trailer ah parkuravana oru like pod...     0
6   en thalaivan yogi babu irukkaar padam vera lev...     0
7             nerkonda parvai sema sema sema trailer      0
8                            ஏய் 96 ஏமாத்த செம பின்னி     0
9         படம் கண்டிப்பாக வெற்றி பெற செம்ம vara level     0
10  correct enga apa military da oodi vilaiyada so...     1
11           dislike pottavan ratham khaki dha savan      0
12  anandhi army hit like ennayum ipdi aakitingale...     0
13  thalaivaa mass nadipin nayagan surya anna fans...     0
14                 படம் கண்டிப்பாக வெற்றியடைய வாழ்த்த     0
15  hit like if anybody laughed during t

In [24]:
new_df_train['text'].isnull().sum()

0

In [25]:
new_df_train.to_csv('tamil_dataset_1_train.csv', index=False)

In [ ]:
#df = pd.read_csv('/kaggle/working/tamil_dataset_1_train.csv')
#df=pd.DataFrame(df)
#df.head(10)

In [26]:
df_test = pd.DataFrame(dataset["validation"])
label_mapping = {
    0: 'Not_offensive',
    1: 'Offensive',
    2: 'Offensive',
    3: 'Offensive',
    4: 'Offensive',
    5: 'not-Tamil',
}
df_test['label'].replace(label_mapping, inplace=True)

label_mapping1 = {
    'Not_offensive':0,
    'Offensive':1,
    'not-Tamil':2,
}
df_test['label'].replace(label_mapping1, inplace=True)
print(df_test.info())
label_counts = df_test['label'].value_counts()
print(label_counts)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4388 entries, 0 to 4387
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    4388 non-null   object
 1   label   4388 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 68.7+ KB
None
0    3193
1    1023
2     172
Name: label, dtype: int64


In [27]:
df_test.head(20)

,text,label
0,Handsome hunk keri vaa thalaivaa,0
1,தென்காசி மாவட்டம் நாடார் சமுதாயம் சார்பாக வாழ்...,0
2,je vous aime bravo pour clip de merde que j éc...,2
3,சிறப்பு..... மேலும் இது போன்ற படைப்புகள் மிக அ...,0
4,Vera level BGM .. semma trailer. 🤞,0
5,என்னடா பண்ணி வச்சிருக்கீங்க ?!!!! அந்த அம்மாவ ...,1
6,"#kaappaan is gonna be a massive hit ,#suriya #...",0
7,250k likes to go.....Yaralam waiting 🥰,0
8,Supar annan trailar semmaya iruku i am waiting...,0
9,Maranam we Luv u Surya ann,0


In [28]:
translator = google_translator()
for index, row in df_test.iterrows(): # Check if emoji is contained in text
    text = str(row['text'])
    found = demoji.findall(text)
    if found:
        # There is an emoji now detect language
        if detect(text) == 'ta': # Text is Tamil
            for key, value in found.items():
                emojiEng = value
                # Translate to Tamil
                transText = translator.translate(emojiEng, lang_tgt='ta', lang_src='en')
                text = text.replace(key, transText + ' ')
        else: # Text is english
            for key, value in found.items():
                text = text.replace(key, value + ' ')

        df_test.at[index, 'text'] = text

In [29]:
df_test.head(20)

,text,label
0,Handsome hunk keri vaa thalaivaa,0
1,தென்காசி மாவட்டம் நாடார் சமுதாயம் சார்பாக வாழ்...,0
2,je vous aime bravo pour clip de merde que j éc...,2
3,சிறப்பு..... மேலும் இது போன்ற படைப்புகள் மிக அ...,0
4,Vera level BGM .. semma trailer. crossed fing...,0
5,என்னடா பண்ணி வச்சிருக்கீங்க ?!!!! அந்த அம்மாவ ...,1
6,"#kaappaan is gonna be a massive hit ,#suriya #...",0
7,250k likes to go.....Yaralam waiting smiling f...,0
8,Supar annan trailar semmaya iruku i am waiting...,0
9,Maranam we Luv u Surya ann,0


In [ ]:
for index, row in df_test.iterrows():
    found = demoji.findall(row['text'])
    if found:
        print('Found exception')

In [30]:
import contractions
# contracted text
for index, row in df_test.iterrows():# creating an empty list
    expanded_words = []
    for word in str(row['text']).split():# using contractions.fix to expand the shotened words
        expanded_words.append(contractions.fix(word))

    expanded_text = ' '.join(expanded_words)
    df_test.at[index, 'text'] = expanded_text


for index, row in df_test.iterrows():
    Sentence = str(row['text'])
    for i in Sentence:
        # checking whether the char is punctuation.
        if i in string.punctuation:
            Sentence = Sentence.replace(i, '')
    df_test.at[index, 'text'] = Sentence.lower()

In [31]:
df_test.head(20)

,text,label
0,handsome hunk keri vaa thalaivaa,0
1,தென்காசி மாவட்டம் நாடார் சமுதாயம் சார்பாக வாழ்...,0
2,je vous aime bravo pour clip de merde que j éc...,2
3,சிறப்பு மேலும் இது போன்ற படைப்புகள் மிக அவசியம்,0
4,vera level bgm semma trailer crossed fingers,0
5,என்னடா பண்ணி வச்சிருக்கீங்க அந்த அம்மாவ,1
6,kaappaan is going to be a massive hit suriya s...,0
7,250k likes to goyaralam waiting smiling face w...,0
8,supar annan trailar semmaya iruku i am waiting...,0
9,maranam we love you surya ann,0


In [32]:
stopwords = adv.stopwords['tamil']
data=[]
for index, row in df_test.iterrows():
    sent = ''
    line = str(row['text'])
    for i in line.split():
        if i not in stopwords:
            sent += i + " "
    data.append({'text': sent, 'label': str(row['label'])})


new_df_test = pd.DataFrame(data)
new_df_test

,text,label
0,handsome hunk keri vaa thalaivaa,0
1,தென்காசி மாவட்டம் நாடார் சமுதாயம் சார்பாக வாழ்...,0
2,je vous aime bravo pour clip de merde que j éc...,2
3,சிறப்பு படைப்புகள் அவசியம்,0
4,vera level bgm semma trailer crossed fingers,0
...,...,...
4383,மிஷ்கின் சினிமாவின் சலித்து போன வட்டத்திற்குள்...,0
4384,sivaji bhajii sapdu petta sweet sapdu serious ...,0
4385,8k dislike sure all vijay fans,1
4386,lady super star manju warrier fans hit like,0


In [33]:
from tqdm import tqdm

def stem_tamil_text(text):
    tokens = text.split()
    stemmed_tokens = []
    for token in tokens:
        analyzed_tokens = analyzer.morph_analyze_document([token])
        if analyzed_tokens:
            stemmed_tokens.append(analyzed_tokens[0])
        else:
            stemmed_tokens.append(token)  # If stemming fails, keep the original token
    return ' '.join(stemmed_tokens)


for index, row in tqdm(new_df_test.iterrows(), total=len(new_df_test)):
    text = str(row['text'])

    try:
        detected_lang = detect(text)
    except LangDetectException as e:
        # Handle the exception, for example by setting the detected_lang to None
        detected_lang = None

    if detected_lang == 'ta':
        stemmed_text = stem_tamil_text(text)
        new_df_test.at[index, 'text'] = stemmed_text


print(new_df_test.head(20))


100%|██████████| 4388/4388 [01:01<00:00, 71.70it/s]

                                                 text label
0                   handsome hunk keri vaa thalaivaa      0
1    தென்காசி மாவட்ட நாடார் சமுதாய சார்பாக வாழ்த்துக்     0
2   je vous aime bravo pour clip de merde que j éc...     2
3                          சிறப்பு படைப்புகள் அவசியம்     0
4       vera level bgm semma trailer crossed fingers      0
5                        என்ன பண்ணி வச்சிருக்க அம்மாவ     1
6   kaappaan is going to be a massive hit suriya s...     0
7   250k likes to goyaralam waiting smiling face w...     0
8   supar annan trailar semmaya iruku i am waiting...     0
9                      maranam we love you surya ann      0
10  absolute hardwork lot of love 4 ilampuyal jaya...     0
11                         sunny leone fans hit like      0
12                 thaka fans ah iru dha like poduga      0
13  இராமநாத மாவட்ட சார்பாக அனைத்து சமுதாய பெண் பாத...     0
14                   love super comedy grand pa love      0
15                              therika 

In [34]:
new_df_test['text'].isnull().sum()

0

In [35]:
new_df_test.to_csv('tamil_dataset_1_test.csv', index=False)